In [2]:
import pandas as pd
import os, glob
import numpy as np
from scipy import stats
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math
from scipy import stats
import numpy.random as rd
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import catboost as ctb
from sklearn.metrics import mean_squared_error as MSE 
from time import time
from numpy import mean
from numpy import std
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
import lightgbm as lgb 
from scipy import stats
from sklearn.tree import DecisionTreeRegressor
from ctgan import CTGAN
import torch
import sklearn.model_selection
import joblib
from sklearn.metrics import mean_absolute_percentage_error
from datetime import datetime

In [7]:
def Declare_file(df_merged):
  merged_file=df_merged                                         
  # new_file=[f for f in merged_file.columns if f not in ["Longitude","Latitude",
  #                                                       "Epicenter Distance","Hypocenter Distance","Station_Lat",
  #                                                 "Station_Lon","Epicenter_Lat","Epicenter_Lon","Original Time",
  #                                                 "Recording Start Time","Station Name","Magnitude",
  #                                                 "Recording Start Time"
                                                  
  #                                                 ]]
  new_file=['Vs30 Actual', 'TAU_P', 'TAU_C', 'PGV UD', 
            'PGD UD', 'pgv/pgd', 'ACF ACC 1 UD NEG', 'ACF ACC 2 UD POS',
            'ACF VEL 1 UD NEG', 'ACF VEL 2 UD POS', 'ACF VEL 2 UD NEG',
            'ACF DISP 1 UD POS', 'ACF DISP 1 UD NEG', 'ACF DISP 2 UD POS',
            'ACF DISP 2 UD NEG', '0 cross 1 ACC', '0 cross 1 VEL', '0 cross 1 DISP', 
            '0 cross 2 ACC', '0 cross 2 DISP', 'ACF RATIO ACC', 'ACF RATIO DISP',
            'VEL_ACF_1', 'VEL_ACF_2', 'VEL_ACF_3', 'VEL_ACF_5', 'VEL_ACF_6',
            'VEL_ACF_7', 'RSSC VELOCITY', 'CA VELOCITY', 'IV^2', 'ID^2',
            'T_VA', 'PI_v']
  new_file.append("Magnitude")
  new_merge = merged_file[new_file]   
  new_merge = new_merge[np.isfinite(new_merge).all(1)]
  new_merge = new_merge.dropna()
  return new_merge

def read_Syn(Old_Test_data_fol_loc,New_Test_data_fol_loc):
  #Reading Test
  test_file=pd.read_csv(Old_Test_data_fol_loc)
  test_df=Declare_file(test_file)
  test_df.to_csv(New_Test_data_fol_loc,index=False) 
  dataset=test_df.values
  t=dataset.shape[1]-1
  X_test_ctf = dataset[:,0:t]
  Y_test_ctf = dataset[:,t]
  return X_test_ctf,Y_test_ctf



def evaluate_result(y_actual, y_pred):
  MAE=mean_absolute_error(y_actual, y_pred)
  print("Mean Absolute Error\n",MAE)
  MSE=mean_squared_error(y_actual, y_pred)
  print("Mean Square Error\n",MSE)
  print("Root Mean Square Error\n",math.sqrt(MSE))
  print("-------------------------------------")


def Averaging_L2(file_loc):
  df=pd.read_csv(file_loc,sep=',')
  actual_test=df['Magnitude']
  LightGBM_NonSyn_test= df['LightGBMNonSyn Pred']
  LightGBM_Syn_test= df['LightGBMSyn Pred']
  GWO_XGB_Syn_test= df['GWOXGBSyn Pred']
  GWO_XGB_NonSyn_test= df['GWOXGBNonSyn Pred']
  XGB_Syn_test= df['XGBSyn Pred']
  XGB_NonSyn_test= df['XGBNonSyn Pred']
  CatBNonSyn_test= df['CatBNonSyn Pred']
  CatBSyn_test= df['CatBSyn Pred']
  RFNonSyn_test= df['RFNonSyn Pred']
  RFSyn_test= df['RFSyn Pred']
  avg_pred=(XGB_NonSyn_test.astype(int)+XGB_Syn_test.astype(int)+
          GWO_XGB_NonSyn_test.astype(int)+CatBSyn_test.astype(int)+
          LightGBM_Syn_test.astype(int)+GWO_XGB_Syn_test.astype(int)+
          LightGBM_NonSyn_test.astype(int)+CatBNonSyn_test.astype(int)
          )/8
  final_mag=[]
  for i in range(0,len(avg_pred)):
    if avg_pred[i]>6.2:
      final_mag.append(avg_pred[i])
      continue
    if avg_pred[i]<3.9 and avg_pred[i]>2.9:
      RF_N=(RFNonSyn_test[i]+RFSyn_test[i])/2
      final_mag.append(np.round(RF_N,1))
      continue
    else:
      temp=(XGB_NonSyn_test[i]+CatBNonSyn_test[i]+GWO_XGB_NonSyn_test[i])/3
      final_mag.append(np.round(temp,1))
      continue
  return final_mag

def mag_mean(mag):
  pred_mag=mag['Magnitude Pred'].values
  sum=0
  size=len(pred_mag)
  for i in range(0,size):
    sum=sum+pred_mag[i]
  mean=sum/size
  return mean

def add_station_name(old_file,new_file,generated_file):
  old_df=pd.read_csv(old_file)
  new_df=pd.read_csv(new_file,sep=',')
  #reset index to remove nan
  old_df.reset_index(drop=True, inplace=True)
  new_df.reset_index(drop=True, inplace=True)
  generated_df=pd.concat([old_df, new_df], axis=1)
  generated_df = generated_df.dropna()
  generated_df.to_csv(generated_file,sep=',')

def PGD_method(df):
  Pd=df['PGD UD'].values
  hy=df['Hypocenter Distance'].values
  M= 0.91*np.log10(Pd)+0.48*np.log10(hy)+5.65
  return np.round(M,1)

def Tauc_method(df):
  Tauc=df['TAU_C'].values
  M=2.94*np.log10(Tauc)+5.3
  return np.round(M,1)
  
def Regression_methods(file_loc):
  df=pd.read_csv(file_loc,sep=',')
  df['PGD Pred Mag']=PGD_method(df)
  df['Tauc Pred Mag']=Tauc_method(df)
  actual_mag=df['Magnitude'].values
  df['Error PGD']=actual_mag-df['PGD Pred Mag'].values
  df['Error Tauc']=actual_mag-df['Tauc Pred Mag'].values
  df['Error ML Pred']=actual_mag-df['Magnitude Pred'].values
  df.to_csv(file_loc,index=False)

def mag_select(folder_loc,EQ1_loc,EQ2_loc,EQ3_loc,EQ4_loc):
  #magn 5.8
  df=pd.read_csv(folder_loc,sep=',')
  #EQ1
  df=df[df['Magnitude'] == 7.3]
  df=df[df['Epicenter_Lat'] == 38.032]
  df=df[df['Epicenter_Lon'] == 143.507]
  df.to_csv(EQ1_loc,index=False)
  #EQ2
  df=pd.read_csv(folder_loc,sep=',')
  df=df[df['Magnitude'] == 7]
  df=df[df['Epicenter_Lat'] == 33.738]
  df=df[df['Epicenter_Lon'] == 130.175]
  df.to_csv(EQ2_loc,index=False)
  #EQ3
  df=pd.read_csv(folder_loc,sep=',')
  df=df[df['Magnitude'] == 5.4]
  df=df[df['Epicenter_Lat'] == 34.872]
  df=df[df['Epicenter_Lon'] == 132.893]
  df.to_csv(EQ3_loc,index=False)
  #EQ4
  df=pd.read_csv(folder_loc,sep=',')
  df=df[df['Magnitude'] == 6.2]
  df=df[df['Epicenter_Lat'] == 44.43]
  df=df[df['Epicenter_Lon'] == 141.21]
  df.to_csv(EQ4_loc,index=False)


def percent_error(actual,pred):
    per = (actual - pred) / actual
    per_err = abs(per)
    print(round(per_err*100, 2),"%")
    
def calc_weight_mag(eq_loc):
    df=pd.read_csv(eq_loc,sep=",")
    df = df.dropna()
    station_time=df['Recording Start Time']
    eathquake_time=df['Original Time']
    mag=df['Magnitude Pred']
    print("Total Magnitudes are:\n",mag.shape)
    #calculating time of origin of earthquake
    date_time = datetime.strptime(eathquake_time[0], "%H:%M:%S")
    a_timedelta = date_time - datetime(1900, 1, 1)
    earth_seconds = a_timedelta.total_seconds()
    print("Earthquake Seconds\n",earth_seconds)
    #start time
    start_set=np.zeros(len(mag))
    for i in range(0,len(mag)):
      date_time = datetime.strptime(station_time[i], "%H:%M:%S")
      a_timedelta = date_time - datetime(1900, 1, 1)
      station_seconds = a_timedelta.total_seconds()
      st_time_diff=station_seconds-earth_seconds
      start_set[i]=st_time_diff
    start_time=np.amin(start_set)
    end_time=np.amax(start_set)
    print("End Time\n", end_time)
    print("Start Time\n", start_time)
    max_j=np.round(end_time/3.0,0)+1
    final_mag=np.zeros(int(max_j))
    count_mag=np.zeros(int(max_j))
    count=0
    for i in range(0,len(mag)):
      st_time_diff=start_set[i]
      for j in range(1,int(max_j)):
        k=3*j
        if st_time_diff <k:
          final_mag[j]=final_mag[j]+mag[i]
          count_mag[j]=count_mag[j]+1
          break
    final_mag=final_mag/count_mag
    #check for nan
    nan_final_mag = np.isnan(final_mag)
    not_nan_final_mag = ~ nan_final_mag
    final_mag = final_mag[not_nan_final_mag]
    weighted_final_mag=0
    unweighted_final_mag=0
    for i in range(0,len(final_mag)):
      w=1.005+0.01*i
      weighted_final_mag=weighted_final_mag+w*final_mag[0]
      unweighted_final_mag=unweighted_final_mag+final_mag[0]
    weighted_final_mag=weighted_final_mag/len(final_mag)
    weighted_final_mag=np.round(weighted_final_mag,1)
    print("Weighted Final Magnitude",weighted_final_mag)
    unweighted_final_mag=unweighted_final_mag/len(final_mag)
    unweighted_final_mag=np.round(unweighted_final_mag,1)
    print("Unweighted result")
    print(unweighted_final_mag)
    print('Actual Magnitude')
    print(df['Magnitude'][0])
    print("Percentage error for unweighted magnitude:")
    percent_error(df['Magnitude'][0],unweighted_final_mag)
    print("Percentage error for weighted magnitude:")
    percent_error(df['Magnitude'][0],weighted_final_mag)
    print("-------------------------------------")


def Statistical_eval(file_loc):
  df=pd.read_csv(file_loc,sep=',')
  actual_test=df['Magnitude']
  LightGBM_NonSyn_test= df['LightGBMNonSyn Pred']
  LightGBM_Syn_test= df['LightGBMSyn Pred']
  GWO_XGB_Syn_test= df['GWOXGBSyn Pred']
  GWO_XGB_NonSyn_test= df['GWOXGBNonSyn Pred']
  XGB_Syn_test= df['XGBSyn Pred']
  XGB_NonSyn_test= df['XGBNonSyn Pred']
  CatBNonSyn_test= df['CatBNonSyn Pred']
  CatBSyn_test= df['CatBSyn Pred']
  RFNonSyn_test= df['RFNonSyn Pred']
  RFSyn_test= df['RFSyn Pred']
  ML_test= df['Magnitude Pred']
  print("MagPred Result\n")
  evaluate_result(actual_test, ML_test)
  RFNonSyn_test= df['PGD Pred Mag']
  print("Predicted Magnitude using PGD regression relation : \n ")
  evaluate_result(actual_test, RFNonSyn_test) 
  RFSyn_test= df['Tauc Pred Mag']
  print("Predicted Magnitude using Tauc regression relation : \n ")
  evaluate_result(actual_test, RFSyn_test)
  ML_test= df['Magnitude Pred']
  print("Predicted Magnitude using MagPred model :\n ")
  evaluate_result(actual_test, ML_test)

#Declaring File locations
Old_Test_data_fol_loc='~/notebook/Paper 6/Work paper 6/Model_Wise_Result/Github Data/Test_Dataset.csv'
New_Test_data_fol_loc='~/notebook/Paper 6/Work paper 6/Model_Wise_Result/Github Data/Test_Dataset_Modified.csv'
Result_Test_fol_loc='~/notebook/Paper 6/Work paper 6/Model_Wise_Result/Github Data/Test_Result.csv'
EQ1_loc='~/notebook/Paper 6/Work paper 6/Model_Wise_Result/Github Data/EQR1.csv'
EQ2_loc='~/notebook/Paper 6/Work paper 6/Model_Wise_Result/Github Data/EQR2.csv'
EQ3_loc='~/notebook/Paper 6/Work paper 6/Model_Wise_Result/Github Data/EQR3.csv'
EQ4_loc='~/notebook/Paper 6/Work paper 6/Model_Wise_Result/Github Data/EQR4.csv'

#Reading the Built file
X_test_ctf,Y_test_ctf=read_Syn(Old_Test_data_fol_loc,New_Test_data_fol_loc)
print('shape of input test set',X_test_ctf.shape)


shape of input test set (49, 34)


In [8]:
XGB_NonSyn=joblib.load('Work paper 6/Models/XGB_NonSyn.pkl')
XGB_Syn=joblib.load('Work paper 6/Models/XGB_Syn.pkl')
CatB_NonSyn=joblib.load('Work paper 6/Models/CatB_NonSyn.pkl')
CatB_Syn=joblib.load('Work paper 6/Models/CatB_Syn.pkl')
LightGBM_NonSyn=joblib.load('Work paper 6/Models/LightGBM_NonSyn.pkl')
LightGBM_Syn=joblib.load('Work paper 6/Models/LightGBM_Syn.pkl')
GWO_XGB_NonSyn=joblib.load('Work paper 6/Models/GWO_XGB_NonSyn.pkl')
GWO_XGB_Syn=joblib.load('Work paper 6/Models/GWO_XGB_Syn.pkl')
RF_1_NonSyn=joblib.load('Work paper 6/Models/RF_1_NonSyn.pkl')
RF_1_Syn=joblib.load('Work paper 6/Models/RF_1_Syn.pkl')


In [9]:
def Testing_on_Dataset(GWO_XGB_NonSyn,GWO_XGB_Syn,XGB_NonSyn,XGB_Syn,CatB_NonSyn,CatB_Syn,LightGBM_NonSyn,LightGBM_Syn,
                       RF_1_NonSyn,RF_1_Syn,X_test_ctf,Y_test_ctf,EQ1_loc,EQ2_loc,EQ3_loc,EQ4_loc,
                       New_Test_data_fol_loc,Result_fol_loc,eq_flag):
  #SAving result in other folder
  df=pd.read_csv(New_Test_data_fol_loc,sep=',')
  df.to_csv(Result_fol_loc,index=False)
  test_df=pd.read_csv(Result_fol_loc,sep=',')
  print("\nRESULT\n")
  y_pred_GWO_XGB_NonSyn=GWO_XGB_NonSyn.predict(X_test_ctf)
  y_pred_GWO_XGB_Syn=GWO_XGB_Syn.predict(X_test_ctf)
  test_df['GWOXGBNonSyn Pred']=np.round(y_pred_GWO_XGB_NonSyn, 1)
  test_df['GWOXGBSyn Pred']=np.round(y_pred_GWO_XGB_Syn, 1)
  y_pred_xgb_NonSyn=XGB_NonSyn.predict(X_test_ctf)
  y_pred_xgb_Syn=XGB_Syn.predict(X_test_ctf)
  test_df['XGBNonSyn Pred']=np.round(y_pred_xgb_NonSyn, 1)
  test_df['XGBSyn Pred']=np.round(y_pred_xgb_Syn, 1)
  y_pred_RF_1_NonSyn=RF_1_NonSyn.predict(X_test_ctf)
  y_pred_RF_1_Syn=RF_1_Syn.predict(X_test_ctf)
  test_df['RFNonSyn Pred']=np.round(y_pred_RF_1_NonSyn, 1)
  test_df['RFSyn Pred']=np.round(y_pred_RF_1_Syn, 1)
  y_pred_CatB_NonSyn=CatB_NonSyn.predict(X_test_ctf)
  y_pred_CatB_Syn=CatB_Syn.predict(X_test_ctf)
  test_df['CatBNonSyn Pred']=np.round(y_pred_CatB_NonSyn, 1)
  test_df['CatBSyn Pred']=np.round(y_pred_CatB_Syn, 1)
  y_pred_LightGBM_NonSyn=LightGBM_NonSyn.predict(X_test_ctf,num_iteration=LightGBM_NonSyn.best_iteration)
  y_pred_LightGBM_Syn=LightGBM_Syn.predict(X_test_ctf,num_iteration=LightGBM_Syn.best_iteration)
  test_df['LightGBMNonSyn Pred']=np.round(y_pred_LightGBM_NonSyn, 1)
  test_df['LightGBMSyn Pred']=np.round(y_pred_LightGBM_Syn, 1)
  test_df.to_csv(Result_fol_loc,index=False)
  final_mag=Averaging_L2(Result_fol_loc)
  final_mag=np.array(final_mag)
  test_df['Magnitude Pred']=np.round(final_mag, 1)
  test_df.to_csv(Result_fol_loc,index=False)
  add_station_name(Old_Test_data_fol_loc,Result_fol_loc,Result_fol_loc)
  Regression_methods(Result_fol_loc)
  mag_select(Result_fol_loc,EQ1_loc,EQ2_loc,EQ3_loc,EQ4_loc)
  Statistical_eval(Result_fol_loc)
  if eq_flag==1:
      #Earthquake 1
      print("EARTHQUAKE 1")
      calc_weight_mag(EQ1_loc)
      #Earthquake 2
      print("EARTHQUAKE 2")
      calc_weight_mag(EQ2_loc)
      #Earthquake 3
      print("EARTHQUAKE 3")
      calc_weight_mag(EQ3_loc)
      #Earthquake 
      print("EARTHQUAKE 4")
      calc_weight_mag(EQ4_loc)

  

In [10]:
#Testing the model
Testing_on_Dataset(GWO_XGB_NonSyn,GWO_XGB_Syn,XGB_NonSyn,XGB_Syn,CatB_NonSyn,CatB_Syn,LightGBM_NonSyn,LightGBM_Syn,
                       RF_1_NonSyn,RF_1_Syn,X_test_ctf,Y_test_ctf,EQ1_loc,EQ2_loc,EQ3_loc,EQ4_loc,
                       New_Test_data_fol_loc,Result_Test_fol_loc,1)


RESULT

MagPred Result

Mean Absolute Error
 0.7142857142857143
Mean Square Error
 0.8016326530612246
Root Mean Square Error
 0.895339406628137
-------------------------------------
Predicted Magnitude using PGD regression relation : 
 
Mean Absolute Error
 2.0693877551020408
Mean Square Error
 4.700816326530612
Root Mean Square Error
 2.168136602368636
-------------------------------------
Predicted Magnitude using Tauc regression relation : 
 
Mean Absolute Error
 2.8020408163265307
Mean Square Error
 8.570816326530613
Root Mean Square Error
 2.9275956562562757
-------------------------------------
Predicted Magnitude using MagPred model :
 
Mean Absolute Error
 0.7142857142857143
Mean Square Error
 0.8016326530612246
Root Mean Square Error
 0.895339406628137
-------------------------------------
EARTHQUAKE 1
Total Magnitudes are:
 (21,)
Earthquake Seconds
 35820.0
End Time
 57.0
Start Time
 36.0
Weighted Final Magnitude 7.0
Unweighted result
6.8
Actual Magnitude
7.3
Percentage erro

/tmp/ipykernel_172666/768276120.py:189: RuntimeWarning: invalid value encountered in true_divide
  final_mag=final_mag/count_mag
/tmp/ipykernel_172666/768276120.py:189: RuntimeWarning: invalid value encountered in true_divide
  final_mag=final_mag/count_mag
/tmp/ipykernel_172666/768276120.py:189: RuntimeWarning: invalid value encountered in true_divide
  final_mag=final_mag/count_mag
/tmp/ipykernel_172666/768276120.py:189: RuntimeWarning: invalid value encountered in true_divide
  final_mag=final_mag/count_mag
